<div style="text-align:right"><i>Peter Norvig<br>Jan 2016<br>revised 2018, 2020</i></div>

# Making Numbers: Countdowns, Four 4s, Five 5s, ...

In this notebook we solve a range of related puzzles that all involve making mathematical expressions by combining numbers and operators in various ways to make target numeric values.  First some imports, and then we can look at the first problem.

In [ ]:
from collections import Counter, defaultdict, namedtuple
from fractions   import Fraction
from functools   import lru_cache
from itertools   import product, permutations
from math        import sqrt, factorial, floor, ceil
from operator    import add, sub, mul, neg, truediv as div
from typing      import List, Tuple, Dict, Union, Sequence, Set, Optional
import re

# Countdown to 2016

On January 1, 2016 Alex Bellos [posed](http://www.theguardian.com/science/2016/jan/04/can-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016) (and subsequently [answered](http://www.theguardian.com/science/2016/jan/04/did-you-solve-it-complete-the-equation-10-9-8-7-6-5-4-3-2-1-2016)) this New Year's puzzle:


> Fill in the blanks so that this equation makes arithmetical sense:
>
> `10 ␣ 9 ␣ 8 ␣ 7 ␣ 6 ␣ 5 ␣ 4 ␣ 3 ␣ 2 ␣ 1 = 2016`
>
> You are allowed to use *only* the four basic arithmetical operations: +, -, &times;, ÷. But brackets (parentheses) can be used wherever needed. So, for example, the solution could begin
>
> `(10 + 9) * (8` ...  or  `10 + (9 * 8)` ...
# Countdown to 2016: Four Operators, No Brackets

We'll start with a  simpler version of the puzzle: a countdown with no brackets.   

There are nine blanks,  each of which can be filled by one of four operators, so there are 9<sup>4</sup> = 262,144 possibilities, few enough that we can enumerate them all, using `itertools.product` to get tuples of operators, and then `str.format` to plug them into blanks, and then `eval` to evaluate the string:

In [2]:
countdown = '10{}9{}8{}7{}6{}5{}4{}3{}2{}1'

allops = list(product('+-*/', repeat=9))

In [3]:
allops[0]

('+', '+', '+', '+', '+', '+', '+', '+', '+')

In [4]:
allops[100003]

('-', '*', '+', '-', '*', '*', '*', '+', '/')

In [5]:
countdown.format(*allops[100003])

'10-9*8+7-6*5*4*3+2/1'

In [6]:
eval(_)

-413.0

(*Warning*: don't use eval on strings that you aren't sure are safe.)

We need to catch errors such as dividing by zero, so I'll define a wrapper function, `evaluate`, to do that: 

In [7]:
Exp = str # The type of expressions, such as "(10+9)"

def evaluate(exp: Exp) -> object:
    """eval(exp), or None if there is an arithmetic error."""
    try:
        return eval(exp)
    except ArithmeticError:
        return None

Now we can try to find an expression that evaluates to 2016:

In [8]:
for ops in allops:
    exp = countdown.format(*ops)
    if evaluate(exp) == 2016:
        print(exp)

Too bad; we did all that work and no solution was printed. What years *can* we find solutions for? I'll define the function `simple_countdowns` to take a collection of target years and return a dict of the form `{year: 'expression'}` for each expression that evaluates to one of the target years. I'll generalize the function to allow any format string, and any collection of Python operators:

In [9]:
def simple_countdowns(targets, operators='+-*/', fmt=countdown) -> Dict[int, Exp]:
    """All solutions to the countdown puzzle (with no brackets) for target years."""
    exps = (fmt.format(*ops)
            for ops in product(operators, repeat=fmt.count('{}')))
    table = {evaluate(exp): exp for exp in exps}
    return {t: table[t] for t in targets if t in table}

simple_countdowns(range(1900, 2100))

{1979: '10*9*8+7*6*5*4*3/2-1',
 1980: '10*9*8+7*6*5*4*3/2/1',
 1981: '10*9*8+7*6*5*4*3/2+1',
 2013: '10*9*8*7/6/5*4*3-2-1',
 2014: '10*9*8*7/6/5*4*3-2/1',
 2015: '10*9*8*7/6/5*4*3-2+1',
 2017: '10*9*8*7/6/5*4*3+2-1',
 2018: '10*9*8*7/6/5*4*3+2/1',
 2019: '10*9*8*7/6/5*4*3+2+1'}

Interesting: in the 20th and 21st centuries, there are only two "golden eras" where the bracketless countdown equation works: the three year period centered on 1980, and the seven year period that is centered on 2016, but omits 2016. 

# Countdown to 2016: Four  Operators, With Brackets

Now we return to the original problem. Can I make use of what I have so far? Well,  `simple_countdowns` accepts a `fmt` string as input, so I could give it all possible bracketed format strings and let it fill in all possible operator combinations. How long would that take? I happen to remember that the number of bracketings of *n* operands is the nth [Catalan number](https://oeis.org/A000108), so for *n* = 9 there are 4,862 bracketings. A single call to `simple_countdown` took about 2 seconds, so we could do 4,862 calls in about 160 minutes. That's not terrible, but (a) it would still take work to generate all the bracketings, and (b) I think I can find another way that is much faster.

I'll restate the problem as: given the ordered sequence of numbers (10, 9, ... 1), find an expression whose value is 2016. But to get there I'll solve a more general problem: given any sequence of numbers, like `(10, 9, 8)`, what `{value: expression}` pairs can I make with them? I'll define  `expressions(numbers)` to return an **expression table**: a dict of `{value: expression}` for all expressions (strings) whose numeric value is `value` that can be made from `numbers`, for example:

    expressions((10,)) ⇒ {10: '10'}
    expressions((9, 8)) ⇒ {1: '(9-8)', 1.125: '(9/8)', 17: '(9+8)', 72: '(9*8)'}

I'll use the idea of [**dynamic programming**](https://en.wikipedia.org/wiki/Dynamic_programming): break the problem down into simpler subparts, compute an answer for each subpart, and remember intermediate results so we don't need to re-compute them later. How do we break the problem into parts? `expressions((10, 9, 8))` should consist of all the ways of splitting `(10, 9, 8)` into two parts, finding all the expressions that can be made with each part, and combining pairs of expressions with any of the four operators:

    expressions((10, 9, 8)) ⇒ {11: '(10+(9-8))', 27: '(10+(9+8))', 720: '(10*(9*8))', ...}

First the function `splits`:

In [10]:
def splits(sequence) -> List[Tuple[Sequence, Sequence]]:
    "Split sequence into two non-empty parts, in all ways."
    return [(sequence[:i], sequence[i:]) 
            for i in range(1, len(sequence))]

In [11]:
c10 = (10, 9, 8, 7, 6, 5, 4, 3, 2, 1) # A countdown from 10 to 1

splits(c10)

[((10,), (9, 8, 7, 6, 5, 4, 3, 2, 1)),
 ((10, 9), (8, 7, 6, 5, 4, 3, 2, 1)),
 ((10, 9, 8), (7, 6, 5, 4, 3, 2, 1)),
 ((10, 9, 8, 7), (6, 5, 4, 3, 2, 1)),
 ((10, 9, 8, 7, 6), (5, 4, 3, 2, 1)),
 ((10, 9, 8, 7, 6, 5), (4, 3, 2, 1)),
 ((10, 9, 8, 7, 6, 5, 4), (3, 2, 1)),
 ((10, 9, 8, 7, 6, 5, 4, 3), (2, 1)),
 ((10, 9, 8, 7, 6, 5, 4, 3, 2), (1,))]

Now the function `expressions`. Note that we place the dummy `@` operator in the `exp` string, and then replace it with real operators in four ways. Note also that `for (L, R) in product(..., ...)` is equivalent to the two lines
     
       for L in ...
           for R in ...
           

In [12]:
ExpTable = Dict[float, Exp] # A table of {1.125: '(9/8)', ...}
    
@lru_cache(None)
def expressions(numbers: tuple) -> ExpTable:
    """Return a {value: exp} table for all expressions that can be made from `numbers`."""
    if len(numbers) == 1: 
        return {numbers[0]: str(numbers[0])}
    else: 
        table = {}
        for (Lnums, Rnums) in splits(numbers):
            for (L, R) in product(expressions(Lnums), expressions(Rnums)):
                exp = '(' + expressions(Lnums)[L] + '@' + expressions(Rnums)[R] + ')'
                if R != 0: 
                    table[L / R] = exp.replace('@', '/')
                table[L * R] = exp.replace('@', '*')
                table[L - R] = exp.replace('@', '-')
                table[L + R] = exp.replace('@', '+')
        return table

For example, at one point in a call to `expressions((10, 9, 8))` we would have:

     Lnums, Rnums          = (10,),  (9, 8)
     L,     R              = 10,     1
     expressions(Lnums)[L] = '10'
     expressions(Rnums)[R] = '(9-8)'
     exp                   = '(10@(9-8))'

This would lead to us adding the following four entries to `table`:

     table[10] = '(10/(9-8))'
     table[10] = '(10*(9-8))'
     table[9]  = '(10-(9-8))'
     table[11] = '(10+(9-8))'
     
The decorator `@lru_cache` takes care of storing the intermediate results so we don't have to recompute them. Rather than catching errors, we just avoid division by 0.

Let's give it a try:

In [13]:
expressions((10,))

{10: '10'}

In [14]:
expressions((9, 8))

{1.125: '(9/8)', 72: '(9*8)', 1: '(9-8)', 17: '(9+8)'}

In [15]:
expressions((10, 9, 8))

{8.88888888888889: '((10/9)*8)',
 11.25: '((10*9)/8)',
 8.875: '(10-(9/8))',
 11.125: '(10+(9/8))',
 0.1388888888888889: '((10/9)/8)',
 720: '((10*9)*8)',
 -62: '(10-(9*8))',
 82: '((10*9)-8)',
 10.0: '(10*(9-8))',
 9: '((10-9)+8)',
 11: '((10+9)-8)',
 0.5882352941176471: '(10/(9+8))',
 170: '(10*(9+8))',
 -7: '((10-9)-8)',
 27: '((10+9)+8)',
 -6.888888888888889: '((10/9)-8)',
 9.11111111111111: '((10/9)+8)',
 98: '((10*9)+8)',
 0.125: '((10-9)/8)',
 8: '((10-9)*8)',
 2.375: '((10+9)/8)',
 152: '((10+9)*8)'}

That looks reasonable. Let's solve the whole puzzle.

# Countdown to 2016: A Solution


In [16]:
%time expressions(c10)[2016]

CPU times: user 27.3 s, sys: 714 ms, total: 28 s
Wall time: 28.6 s


'(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)+1)'

We have an answer! And in seconds, not hours, thanks to dynamic programming! Here are solutions for nearby years:

In [17]:
{y: expressions(c10)[y] for y in range(2010, 2026)}

{2010: '((((((10+(((9*8)-7)*6))*5)+4)+3)+2)+1)',
 2011: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)-1)',
 2012: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)*1)',
 2013: '((((((10+9)*8)+7)*(6+(5*(4/3))))-2)+1)',
 2014: '(((((((10+((9*8)*7))-6)-5)*4)+3)-2)+1)',
 2015: '(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)*1)',
 2016: '(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)+1)',
 2017: '(((10*(((9*8)-((7*6)/(5+4)))*3))-2)-1)',
 2018: '(((10*(((9*8)-((7*6)/(5+4)))*3))-2)*1)',
 2019: '(((10*(((9*8)-((7*6)/(5+4)))*3))-2)+1)',
 2020: '(((((10+((9+((8+7)*6))*5))*4)+3)-2)-1)',
 2021: '((((((((10-9)+(8*7))*6)-5)*4)*3)/2)-1)',
 2022: '((((((((10-9)+(8*7))*6)-5)*4)*3)/2)*1)',
 2023: '(((((10*(((9*8)*(7-6))-5))+4)*3)+2)-1)',
 2024: '(((((10*(((9*8)*(7-6))-5))+4)*3)+2)*1)',
 2025: '(((((10*(((9*8)*(7-6))-5))+4)*3)+2)+1)'}

# Counting Solutions

Alex Bellos had another challenge:  

> I was half hoping a computer scientist would let me know exactly how many solutions [to the Countdown problem] there are with only the four basic operations. Maybe someone will. 

As it stands, my program can't answer that question, because I only keep one expression for each value. 

Also, I'm not sure what it means to be a distinct solution. For example, are `((10+9)+8)` and `(10+(9+8))` different, or are they same, because they both are equivalent to `(10+9+8)`? Similarly, are `((3-2)-1)` and `(3-(2+1))` different, or the same because they both are equivalent to `(3 + -2 + -1)`? I think the notion of "distinct solution" is just inherently ambiguous. My choice is to count each of these as distinct: every expression has exactly ten numbers, nine operators, and nine pairs of brackets, and if an expression differs in any character, it is different. But I won't argue with anyone who prefers a different definition of "distinct solution."

So how can I count expressions? One approach would be to go back to enumerating every equation (all 4862 &times; 4<sup>9</sup> = 1.2 bilion of them) and checking which ones equal 2016. That would take about 40 hours with my Python program. A better approach is to mimic `expressions`, but to make a table of counts, rather than expression strings. I want:

    expression_counts((10, 9, 8))[27] == 2
    
because there are 2 ways to make 27 with the numbers `(10, 9, 8)`, namely, `((10+9)+8)` and `(10+(9+8))`. And in general, if there are `Lcount` ways to make `L` using `Lnums` and `Rcount` ways to make `R` using `Rnums` then there are `Lcount * Rcount` ways of making `L + R` using `Lnums` followed by `Rnums`. So we have:


In [18]:
@lru_cache(None)
def expression_counts(numbers: tuple) -> Counter:
    "Return a Counter of {value: count} for every value that can be made from numbers."
    if len(numbers) == 1: # Only one way to make an expression out of a single number
        return Counter(numbers)
    else: 
        table = Counter()
        for (Lnums, Rnums) in splits(numbers):
            for L, R in product(expression_counts(Lnums), expression_counts(Rnums)):
                count = expression_counts(Lnums)[L] * expression_counts(Rnums)[R]
                if R != 0:
                    table[L / R] += count
                table[L + R] += count
                table[L - R] += count
                table[L * R] += count
        return table

In [19]:
expression_counts((2, 2)) # corresponds to {0: '2-2', 1.0: '2/2', 4: '2+2' or '2*2'}

Counter({1.0: 1, 4: 2, 0: 1})

In [20]:
expression_counts((10, 9, 8))[27] # (10+9)+8 or 10+(9+8)

2

Looks good to me. Now let's see if we can answer Alex's question.

In [21]:
expression_counts(c10)[2016]

30066

This says there are 30,066 distinct expressions for 2016. Is that the answer Alex wanted?

# The Trouble with Round-off Error

I don't think it is the right answer. The trouble is: round-off error.

Let's find all the values in `expressions(c10)` that are very near to `2016`, within ±0.0000000001:

In [22]:
{y: expressions(c10)[y]
 for y in expressions(c10)
 if abs(y - 2016) < 1e-10}

{2016.0000000000002: '((((10*((9*(8+7))-(6/(5+4))))*3)/2)+1)',
 2016.0: '(((((((((10*9)+8)*7)-6)-5)-4)*3)+2)+1)',
 2015.9999999999995: '((((((10*9)*8)*7)*((6/5)-(4-3)))*2)*1)',
 2015.999999999997: '(((10*9)*8)/(7-(6-(((5/(4+3))/2)-1))))',
 2016.0000000000005: '(((((10*9)*8)*((((7*6)/5)-4)-3))*2)*1)',
 2016.0000000000018: '((((((10*9)*8)*7)*(((6/5)-4)+3))*2)*1)',
 2016.0000000000023: '(10*((9*8)/((7-(6-((5/(4+3))/2)))-1)))',
 2015.9999999999998: '((10*((9*(((8-(7/6))*5)-(4*3)))+2))+1)',
 2015.9999999999993: '(10*(((((9*8)*7)*6)/5)*(((4/3)-2)+1)))',
 2016.000000000002: '(((10*9)*8)/((7-(6-((5/(4+3))/2)))-1))',
 2015.999999999999: '((((10*9)*(8+7))-6)/(((5-(4/3))-2)-1))'}

I suspect that all of these actually should be *exactly* equal to 2016. To determine if they are, I could re-do *all* the calculations using exact rational arithmetic, as provided by the `fractions.Fraction` class. From experience I know that would be at least an order of magnitude slower, so instead I'll just verify the small set of expressions in the output above. I'll define `exact(exp)` to return a string that, when passed to `eval`, will exactly calculate `exp` using rational arithmetic:

In [23]:
def exact(exp) -> Exp: return re.sub(r"([0-9]+)", r"Fraction(\1)", exp)

exact('1/(5-2)')

'Fraction(1)/(Fraction(5)-Fraction(2))'

In [24]:
assert eval(exact('1/(5-2)')) == Fraction(1, 3)

Now I can count up all the expressions in the `expression_counts(c10)` table that are near 2016, but with `exact` computation to check that the expressions evaluate to exactly 2016:

In [25]:
sum(expression_counts(c10)[y] 
    for y, exp in expressions(c10).items()
    if abs(y - 2016) < 1e-10 and eval(exact(exp)) == 2016)

44499

I can claim that the answer is  **44,499**, but I would feel more confident if I did *all* the computations with exact arithmetic, and if the result was independently verified by someone else, and passed an extensive test suite. And of course, if you have a different definition of "distinct solution," you will get a different answer; there are multiple possible definitions that are all very reasonable.

# Four 4s

Alex Bellos continued his original puzzle column with a related puzzle:
    
> The most famous “fill in the gaps in the equation” puzzle is known as  [**four fours**](https://en.wikipedia.org/wiki/Four_fours), because every equation is of the form
>
> `4 ␣ 4 ␣ 4 ␣ 4 = x.`
>
>In the classic form of the puzzle you must find a solution for x = 0 to 9 using just addition, subtraction, multiplication and division (and brackets).

This puzzle goes back to a [1914 publication](https://archive.org/details/mathematicalrecr00ball) by the mathematician/magician [W. W. Rouse Ball](https://en.wikipedia.org/wiki/W._W._Rouse_Ball). The solution is easy:

In [26]:
{i: expressions((4, 4, 4, 4))[i] for i in range(10)}

{0: '(((4-4)-4)+4)',
 1: '(((4/4)-4)+4)',
 2: '((4/(4+4))*4)',
 3: '(((4+4)+4)/4)',
 4: '(((4-4)*4)+4)',
 5: '(((4*4)+4)/4)',
 6: '(((4+4)/4)+4)',
 7: '((4-(4/4))+4)',
 8: '(((4+4)/4)*4)',
 9: '(((4/4)+4)+4)'}

Note that I didn't do anything special to take advantage of the fact that in `(4, 4, 4, 4)` the digits are all the same. Happily, `lru_cache` does that for me automatically! If I split that into `(4, 4)` and `(4, 4)`, when it comes time to do the second half of the split, the result will already be in the cache, and so won't be recomputed.

# New Mathematical Operations

Bellos then writes:
    
> If you want to show off, you can introduce **new mathematical operations** such as powers, square roots, concatenation and decimals, ... or use the factorial symbol, `!`.

It seems there are a lot of puzzles that are similar to four 4s, and they all have different rules for which numbers are required and which operators are allowed. To facilitate solving a range of puzzles with different operators, I will redefine `expressions` to take a second argument, `ops`, listing the allowable operations.

The operations will be specified as a string of one-character codes. The table below gives the allowable codes, binary operators on the left and unary operators on the right, except that the last row gives two pseudo-operations  on digits, decimal points and concatenation:

|Code|Operator|Code|Operator|
|------|--------|------|--------|
|`'+'`|addition: 1 + 2 = 3      |`'_'`|unary minus: -2 = -2|
|`'-'`|subtraction: 3 - 2 = 1   |`'√'`|square root: √9 = 3|
|`'*'`|multiplication: 2 * 3 = 6|`'!'`|factorial: 4! = 24 |
|`'/'`|division: 6 / 3 = 2      |`'⌊'`|floor:⌊4.4⌋ = 4|
|`'^'`|exponentiation: 2 ^ 3 = 8|`'⌈'`|ceiling: ⌈4.4⌉ = 5|
|`'.'`|decimal point: 1.23 = 1.23|`'$'`|concatenation of digits: 123 = 123|

I will define the data type `Operator` to define an operator, giving its code symbol, its arity (binary or unary), the Python function to call to do the calculation, the format string for unary operations, and for some operations, a `guard` that says when it is applicable (for example, division is applicable when the second argument is not zero, and for square root, factorial, and exponentiation I impose some limits that are not mathematically necessary, but keep the table from being overpopulated with numbers that are unlikely to help lead to solutions). For binary operations, the code symbol is used to construct the expression string, but for uunary ops the `fmt` field gives a format string; this allows us to have prefix and postfix operators. The function `operators` picks out the operators you want from the `OPERATORS` global variable. You can augment this with new operators as you wish.

In [27]:
def true(*args): return True

Operator = namedtuple('Operator', 'symbol, func, fmt, guard', defaults=[None, true])

OPERATORS = {
    2: {Operator('+', add),
        Operator('-', sub),
        Operator('*', mul),
        Operator('/', div,       None,  lambda L, R: R != 0),
        Operator('^', pow,       None,  lambda L, R: -10 <= R <= 10 and (L > 0 or R == int(R)) 
                                                     and not (L == 0 == R))},
    1: {Operator('√', sqrt,      '√{}', lambda v: 0 < v <= 100 and (120. * v).is_integer()),
        Operator('!', factorial, '{}!', lambda v: v in range(7)),
        Operator('_', neg,       '-{}'),
        Operator('⌊', floor,     '⌊{}⌋'),
        Operator('⌈', ceil,      '⌈{}⌉')}}

OPS = '+-*/^_√!.$' # Default set of operators; omits floor and ceiling.

def operators(arity: int, ops: str) -> List[Operator]:
    """All the operators in OPERATORS with given arity whose code symbol is one of `ops`."""
    return [op for op in OPERATORS[arity] if op.symbol in ops]

There are some complications; here's how I'll handle them:

- **Irrationals**: `√2` is an irrational number (I'll use floating point approximations for everything).
- **Imaginaries**: `√-1` is an imaginary number (I won't allow imaginary numbers).
- **Overflow**: `(10. ^ (9. ^ 8.))`, as a `float`, gives an `OverflowError` (I'll drop overflow results).
- **Memory**: `(10 ^ (9 ^ (8 ^ 7)))`, as an `int`, gives an `OutOfMemoryError` (I'll limit exponents).
- **Infinite unary operators**: `√√√√√√...(4!!!!!!!!...)` (I'll only allow two unary operators per expression).
- **Round-off error**:`(49*(1/49)) == 0.9999999999999999`, not `1` (I'll try to round off).

I'll define the function `do` to do an arithmetic computation, catch any errors, and try to correct round-off errors. The idea is that since my expressions start with integers, results that are close to an integer probably are that integer. So I'll correct `(49*(1/49))` to be `1.0`. Of course, an expression like `(1+(10^-99))` is also very close to `1.0`, but it should not be rounded off. Instead, I'll try to avoid such expressions by silently dropping any value whose magnitude is outside the range of 10<sup>-10</sup> to 10<sup>10</sup> (except of course I will accept an exact 0).

In [29]:
def do(operator, *args) -> Optional[float]: 
    "Return op(*args), trying to correct for roundoff error, or `None` if too big/small/erroneous."
    if operator.guard(*args):
        try:
            val = operator.func(*args)
            return (val        if val == 0 else
                    None       if isinstance(val, complex) else
                    None       if not (1e-10 < abs(val) < 1e10) else
                    round(val) if abs(val - round(val)) < 1e-12 else 
                    val)
        except (ArithmeticError, ValueError):
            pass
    return None

# Refactoring `expressions`

I'll take this opportunity to refactor `expressions` to use the new `OPERATORS`. I introduce  these subfunctions:
- `digit_expressions(digits, ops)`: returns a table of expressions with just digits (and maybe a decimal place). If `'$'` is in `ops` allow concatenation of digits (e.g. `{44: '44'}`) and if `'.'` is in ops allow decimals (e.g. `{4.4: '4.4'}`. 
- `add_unary_expressions(table, ops)`: add expressions like `√4` and `4!` to `table` and return `table`. 
- `add_binary_expressions(table, numbers, ops)`: add expressions like `4+√4` and `4^4` to `table` and return `table`.



In [30]:
@lru_cache(None)
def expressions(numbers: Tuple[int], ops=OPS) -> ExpTable:
    "Return {value: expr} for all expressions that can be made from numbers using ops."
    table = digit_expressions(numbers, ops)
    table = add_binary_expressions(table, numbers, ops)
    return  add_unary_expressions(table, ops)

def digit_expressions(digits: Tuple[int], ops: str) -> ExpTable:
    "All ways of making numbers from these digits (in order), and a decimal point."
    D = ''.join(map(str, digits))
    table = {}
    if '.' in ops: table.update({float(d): d for d in decimals(D)})
    if '$' in ops or len(digits) == 1: table[int(D)] = D
    return table

def decimals(digits: str)-> List[str]:
    """All ways to insert a decimal point into digits."""
    return [digits[:i] + '.' + digits[i:]
            for i in range(len(digits))
            if i <= 1 or not digits.startswith('0')]
      
def add_binary_expressions(table: ExpTable, numbers: tuple, ops: str) -> ExpTable:
    "Add binary expressions by splitting numbers and combining with an op."
    binary_ops = operators(2, ops)
    for (Lnums, Rnums) in splits(numbers):
        for (L, R) in product(expressions(Lnums, ops), expressions(Rnums, ops)):
            exp = '(' + expressions(Lnums, ops)[L] + '@' + expressions(Rnums, ops)[R] + ')'
            for op in binary_ops:
                assign(table, do(op, L, R), exp.replace('@', op.symbol))
    return table
                
def add_unary_expressions(table: ExpTable, ops: str, nesting_level=2) -> ExpTable:
    "Add unary expressions (possibly -v, √v and v!) to table"
    unary_ops = operators(1, ops)
    for _ in range(nesting_level):
        for v in tuple(table):
            for op in unary_ops:
                assign(table, do(op, v), op.fmt.format(table[v]))
    return table

The function `assign` adds a `{val: exp}` entry to `table`, but if there is already an entry for `val`, it prefers the entry with the lowest total *weight*: the number of characters plus extra points for particularly complex characters. The idea is to prefer simpler expressions, but the weights are completely arbitrary. If you prefer complex expressions, you can change the weights to be negative. The weights don't change how many different values can be made, they just change which of two or more expressions are chosen to represent a value.

In [31]:
def assign(table: dict, val: float, exp: str): 
    "Assign table[val] = exp, unless we already have a lighter exp or val is None."
    if val is not None and (val not in table or weight(exp) < weight(table[val])): 
        table[val] = exp
        
WEIGHTS = Counter({'√':2, '!':2, '.':1, '^':1, '/':0.2, '-':0.1, '⌊': 2, '⌈':2})
        
def weight(exp: str) -> int: return len(exp) + sum(WEIGHTS[c] for c in exp)

That's a lot of new code; let's have some tests:

In [32]:
assert digit_expressions((1, 2), OPS) == {12: '12', 0.12: '.12', 1.2: '1.2'}
assert digit_expressions((1, 2), '$') == {12: '12'}
assert digit_expressions((1, 2), '') == {} 
assert digit_expressions((1,), '$') == {1: '1'}

assert decimals('123') == ['.123', '1.23', '12.3']
assert decimals('007') == ['.007', '0.07']

assert add_unary_expressions({16: '16'}, '_') == {16: '16', -16: '-16'}
assert add_unary_expressions({16: '16'}, OPS) == {16: '16', -16: '-16', 4: '√16', -4: '-√16', 
                                                  2: '√√16', 24: '√16!'}

assert expressions((3, 2), '+-*$') == {32.0: '32', 5: '(3+2)', 1: '(3-2)', 6: '(3*2)'}

I'll define a function to print a table of consecutive integers (starting at 0) that can be made by a sequence of numbers:

In [33]:
def show(numbers: tuple, limit=None, ops=OPS, clear=True):
    """Print expressions for integers from 0 up to limit or the first unmakeable integer."""
    if clear: expressions.cache_clear() # To free up memory
    table = expressions(numbers, ops)
    print(f'Can make 0 to {unmakeable(table)-1} with expressions({numbers}, "{ops}").'
          f' ({len(table):,} table entries)\n')
    for i in range(limit or unmakeable(table)): 
        print('{:4} = {}'.format(i, unbracket(table[i])))
        
def unmakeable(table) -> int:
    """The integer i such that table makes every integer from 0 to i - 1, but not i."""
    for i in range(len(table) + 1):
        if i not in table:
            return i
              
def unbracket(exp: str) -> str:
    "Strip outer parens from exp, if they are there"
    return (exp[1:-1] if exp.startswith('(') and exp.endswith(')') else exp)

# Four 4s with New Mathematical Operations

In [34]:
%time show((4, 4, 4, 4))

Can make 0 to 72 with expressions((4, 4, 4, 4), "+-*/^_√!.$"). (705,959 table entries)

   0 = 44-44
   1 = 44/44
   2 = 4*(4/(4+4))
   3 = (4+(4+4))/4
   4 = 4+(4*(4-4))
   5 = (4+(4*4))/4
   6 = √(44-(4+4))
   7 = (44/4)-4
   8 = 4+(4+(4-4))
   9 = 4+(4+(4/4))
  10 = 44/4.4
  11 = 44/√(4*4)
  12 = (4+44)/4
  13 = √4+(44/4)
  14 = 4+(4+(4+√4))
  15 = 4+(44/4)
  16 = .4*(44-4)
  17 = (4/4)+(4*4)
  18 = (44/√4)-4
  19 = 4!-(4+(4/4))
  20 = 4*(4+(4/4))
  21 = (4+4.4)/.4
  22 = √4*(44/4)
  23 = (√4+44)/√4
  24 = 4+(4+(4*4))
  25 = (4+(4*4!))/4
  26 = 4+(44/√4)
  27 = 4+(4!-(4/4))
  28 = 44-(4*4)
  29 = 4+(4!+(4/4))
  30 = (4+(4+4))/.4
  31 = 4!+((4+4!)/4)
  32 = (4*4)+(4*4)
  33 = 4!+((4-.4)/.4)
  34 = 44-(4/.4)
  35 = 4!+(44/4)
  36 = 44-(4+4)
  37 = ((.4+4!)/.4)-4!
  38 = 44-(4+√4)
  39 = 44-(√4/.4)
  40 = 44-√(4*4)
  41 = (.4+(4*4))/.4
  42 = √4+(44-4)
  43 = 44-(4/4)
  44 = 4+(44-4)
  45 = 44+(4/4)
  46 = 4+(44-√4)
  47 = 4!+(4!-(4/4))
  48 = 4*(4+(4+4))
  49 = 44+(√4/.4)
  50 = 4+(√4

We can also solve the "2016 with four fours" puzzle:

In [35]:
expressions((4, 4, 4, 4), OPS)[2016]

'(4!*(4!+(4!/.4)))'

In a [separate video](https://www.youtube.com/embed/Noo4lN-vSvw), Alex Bellos shows  how to form **every** integer from 0 to infinity using four 4s, if the square root and `log` functions are allowed. The solution comes from Paul Dirac (although [Dirac originally developed it](https://nebusresearch.wordpress.com/2014/04/18/how-dirac-made-every-number/) for the "four 2s" problem).

Donald Knuth [conjectured](https://www.tandfonline.com/doi/abs/10.1080/0025570X.1964.11975546) that with floor, square root and factorial, you can make any positive integer with just **one** 4.

Below are some popular variant problems:

# Four 2s

In [36]:
show((2, 2, 2, 2))

Can make 0 to 30 with expressions((2, 2, 2, 2), "+-*/^_√!.$"). (109,747 table entries)

   0 = 22-22
   1 = 22/22
   2 = 2+(2*(2-2))
   3 = (2+(2*2))/2
   4 = .2*(22-2)
   5 = 2+(2+(2/2))
   6 = (2*(2*2))-2
   7 = 2+(2/(.2*2))
   8 = 2+(2+(2*2))
   9 = (22/2)-2
  10 = 22/2.2
  11 = 22/√(2*2)
  12 = (2+22)/2
  13 = 2+(22/2)
  14 = (2^(2*2))-2
  15 = (2+(2/2))/.2
  16 = 2*(2*(2*2))
  17 = 22-(√.2^-2)
  18 = 22-(2*2)
  19 = (2+(2-.2))/.2
  20 = 22-√(2*2)
  21 = 22-(2/2)
  22 = 2+(22-2)
  23 = 22+(2/2)
  24 = 22+√(2*2)
  25 = (2-2.2)^-2
  26 = 2+(2+22)
  27 = 22+(√.2^-2)
  28 = 2+(2+(2*2)!)
  29 = 2+(2+(.2^-2))
  30 = (2+(2*2))/.2


# Four 9s

In [37]:
show((9, 9, 9, 9))

Can make 0 to 61 with expressions((9, 9, 9, 9), "+-*/^_√!.$"). (539,849 table entries)

   0 = 99-99
   1 = 99/99
   2 = (99/9)-9
   3 = (9+(9+9))/9
   4 = (9+(9*√9))/9
   5 = √9+((9+9)/9)
   6 = √(9+(9+(9+9)))
   7 = 9-((9+9)/9)
   8 = (99/9)-√9
   9 = 9+(9*(9-9))
  10 = 99/9.9
  11 = 9+((9+9)/9)
  12 = (9+99)/9
  13 = 9+(√9+(9/9))
  14 = √9+(99/9)
  15 = 9+((9+9)/√9)
  16 = 9+((9/.9)-√9)
  17 = 9+(9-(9/9))
  18 = 99-(9*9)
  19 = 9+(9+(9/9))
  20 = 9+(99/9)
  21 = (9+9.9)/.9
  22 = 9+(√9+(9/.9))
  23 = √9+((9+9)/.9)
  24 = (99/√9)-9
  25 = 9+(√9!+(9/.9))
  26 = (9*√9)-(9/9)
  27 = 9+(9+√(9*9))
  28 = 9+(9+(9/.9))
  29 = 9+((9+9)/.9)
  30 = (99-9)/√9
  31 = (99-√9!)/√9
  32 = (99-√9)/√9
  33 = √9*(99/9)
  34 = (√9+99)/√9
  35 = (√9!+99)/√9
  36 = 9+(9+(9+9))
  37 = (9/.9)+(9*√9)
  38 = √9!*(√9+(√9/.9))
  39 = 9+(9*(√9/.9))
  40 = (9+(9*√9))/.9
  41 = (.9+(√9!*√9!))/.9
  42 = 9+(99/√9)
  43 = √9+(√9!*(√9!/.9))
  44 = (9*√9!)-(9/.9)
  45 = 9+(9+(9*√9))
  46 = (9/.9)+(√9!*√9!)
  47 = √9*(

# Four 5s

In [38]:
%time show((5, 5, 5, 5))

Can make 0 to 38 with expressions((5, 5, 5, 5), "+-*/^_√!.$"). (202,937 table entries)

   0 = 55-55
   1 = 55/55
   2 = 5!/(5+55)
   3 = (5+(5+5))/5
   4 = √(5+(55/5))
   5 = 5+(5*(5-5))
   6 = (55/5)-5
   7 = 5+((5+5)/5)
   8 = 5.5+(.5*5)
   9 = 5+(5-(5/5))
  10 = 55/5.5
  11 = 5.5+5.5
  12 = (5+55)/5
  13 = (5!-55)/5
  14 = (5!/5)-(5+5)
  15 = (5*5)-(5+5)
  16 = 5+(55/5)
  17 = 5+(5!/(5+5))
  18 = ((5!-5)/5)-5
  19 = (5!-(5*5))/5
  20 = 5+(5+(5+5))
  21 = (5+5.5)/.5
  22 = 55/(.5*5)
  23 = 55-(.5^-5)
  24 = (5*5)-(5/5)
  25 = .5*(55-5)
  26 = (5/5)+(5*5)
  27 = (.5*55)-.5
  28 = .5+(.5*55)
  29 = (5!+(5*5))/5
  30 = 55-(5*5)
  31 = 55-(5!/5)
  32 = (5.5-5)^-5
  33 = .5*(5!*.55)
  34 = 5+(5+(5!/5))
  35 = 5+(5+(5*5))
  36 = (5!+(.5*5!))/5
  37 = 5+(.5^-√(5*5))
  38 = ((5!/5)-5)/.5
CPU times: user 8.6 s, sys: 19.4 ms, total: 8.62 s
Wall time: 8.63 s



# Countdown to 2018

Now another type of New Year's countdown puzzle. On December 31 2017, [Michael Littman](http://cs.brown.edu/~mlittman/) posted this:

> 2+0+1×8, 2+0-1+8, (2+0-1)×8, |2-0-1-8|, -2-0+1×8, -(2+0+1-8), √|2+0-18|, 2+0+1^8, 20-18, 2^(0×18), 2×0×1×8... Happy New Year!

Can we replicate that countdown? For 2018 and for following years?

In [39]:
show((2,0,1,8), 11)

Can make 0 to 30 with expressions((2, 0, 1, 8), "+-*/^_√!.$"). (50,633 table entries)

   0 = 2*(0*18)
   1 = 2^(0*18)
   2 = 20-18
   3 = 2+(01^8)
   4 = √(-2+018)
   5 = -(2+01)+8
   6 = √(2*018)
   7 = -2+(01+8)
   8 = (2-01)*8
   9 = (2-01)+8
  10 = 2+(01*8)


In [40]:
show((2,0,1,9), 11)

Can make 0 to 36 with expressions((2, 0, 1, 9), "+-*/^_√!.$"). (72,371 table entries)

   0 = 2*(0*19)
   1 = 20-19
   2 = 2+(0*19)
   3 = 2+(01^9)
   4 = (2-01)+√9
   5 = 2+(01*√9)
   6 = -(2+01)+9
   7 = -2+(01*9)
   8 = -2+(01+9)
   9 = (2-01)*9
  10 = 20-(1+9)


In [41]:
show((2,0,2,0), 11)

Can make 0 to 27 with expressions((2, 0, 2, 0), "+-*/^_√!.$"). (8,195 table entries)

   0 = 202*0
   1 = 20/20
   2 = 2+(0*20)
   3 = 2+(02^0)
   4 = .2*020
   5 = 2+(02+0!)
   6 = 2*(02+0!)
   7 = 2+(0!/.20)
   8 = 2^(02+0!)
   9 = (20/2)-0!
  10 = 2/0.20


In [42]:
show((2,0,2,1), 11)

Can make 0 to 33 with expressions((2, 0, 2, 1), "+-*/^_√!.$"). (38,867 table entries)

   0 = 2*(0*21)
   1 = -20+21
   2 = 2+(0*21)
   3 = 2+(02-1)
   4 = 2*(02*1)
   5 = 2+(02+1)
   6 = 2*(02+1)
   7 = 2+(0.2^-1)
   8 = 2^(02+1)
   9 = (20/2)-1
  10 = 20/(2*1)


In [43]:
show((2,0,2,2), 11)

Can make 0 to 32 with expressions((2, 0, 2, 2), "+-*/^_√!.$"). (43,815 table entries)

   0 = 2*(0*22)
   1 = 2-(02/2)
   2 = -20+22
   3 = 2+(02/2)
   4 = 2*((0*2)+2)
   5 = 20/(2*2)
   6 = 2+(02*2)
   7 = 2+(0!+(2*2))
   8 = 2*(02*2)
   9 = (20-2)/2
  10 = 20-(2/.2)


# Making 6 from 3 Digits

Nicolas Schank posted the following in the Facebook "omg math" group.

> For each digit from 0 to 9, find at least one way to express 6 using only that digit exactly three times and arithmetic operations. For instance, using the digit 2, `'2+2^2'` = 6.

It turns out this is easy (if we assume "arithmetic operations" include square root and factorial):

In [44]:
[expressions((n, n, n), '+-*/^!√').get(6) for n in range(10)]

['(0!+(0!+0!))!',
 '(1+(1+1))!',
 '(2+(2*2))',
 '((3*3)-3)',
 '(4+(4-√4))',
 '(5+(5/5))',
 '(6+(6-6))',
 '(7-(7/7))',
 '(8-√√(8+8))',
 '((9+9)/√9)']

# The 24 Problem

In the [538 Riddler for July 10th 2020](https://fivethirtyeight.com/features/can-you-make-24/), Zach Wissner-Gross asks "Can you make 24?" from the digits (2, 3, 3, 4), in any order, with just the five binary operators. For extra credit, Zach asks for multiple ways to make 24. Readers sent in suggestions for other sets of four digits with which to make 24.

We haven't dealt with reporting multiple ways, and we haven't allowed different orders for the digits. I'll deal with both by trying all permutations of the digits, and collecting one expression from each permutation (if there is one). If there are multiple ways with a single permutation we won't get more than one of those, but this approach should be good enough to answer the question.

In [45]:
def can_make(total: int, numbers: tuple, ops=OPS) -> Set[Exp]:
    """Can we make the total from the numbers (in any order)? Return a set of ways to do it."""
    return {expressions(nums, ops).get(total)
            for nums in set(permutations(numbers))} - {None}

can_make(24, (2, 3, 3, 4), '+-*/^')

{'(((3^2)-3)*4)',
 '(((4-2)^3)*3)',
 '(3*((4-2)^3))',
 '(3*(4^(3/2)))',
 '(3/((2/4)^3))',
 '(4*((3^2)-3))'}

In [46]:
can_make(25, (2, 3, 8, 8), '+-*/^')

{'((3^2)+(8+8))', '(8+((3^2)+8))', '(8+(8+(3^2)))'}

In [47]:
can_make(24, (2, 3, 10, 10), '+-*/^')

{'(((10-3)*2)+10)',
 '((2*(10-3))+10)',
 '((2^10)-(10^3))',
 '(10+((10-3)*2))',
 '(10+(2*(10-3)))',
 '(10-((3-10)*2))',
 '(10-(2*(3-10)))'}

In [48]:
can_make(24, (3, 3, 8, 8), '+-*/^')

{'(8/(3-(8/3)))'}

In [49]:
can_make(24, (2, 3, 6, 6), '+-*/^')

{'(((3*6)-6)*2)',
 '(((3+2)*6)-6)',
 '(((3+6)*2)+6)',
 '(((6+3)*2)+6)',
 '((2*(3+6))+6)',
 '((2*(6+3))+6)',
 '((2+6)*(6-3))',
 '(6*(6*(2/3)))',
 '(6+((3+6)*2))',
 '(6+((6+3)*2))',
 '(6+(2*(3+6)))',
 '(6+(2*(6+3)))'}

In [50]:
can_make(24, (0, 0, 2, 5), '+-*/^')

set()

I didn't get an answer for that one. I can allow factorial:

In [51]:
can_make(24, (0, 0, 2, 5), '+-*/^!')

{'(((2*0)-0!)+5)!',
 '((0*2)+(5-0!)!)',
 '((0-(2^0))+5)!',
 '((0-0!)+(5^2))',
 '((2*0)+(5-0!)!)',
 '((5-0!)!+(0*2))',
 '((5^(0+2))-0!)',
 '((5^2)+(0-0!))',
 '(0+((0!-2)+5)!)',
 '(0+((5^2)-0!))',
 '(0+(5+(0!-2))!)',
 '(2*((5^0)+0!))!'}

# Even More Fours: The Power of Floor and Ceiling

With the standard set of `OPS`, we got all the integers up to 72 with four 4s. If we add the floor and ceiling operators, we get a big jump: suddenly, all the results of a division or a square root that didn't form an integer can now be coerced into integers. Instead of getting the integers only up to 72, we now get all the way up to 1644 (although it takes three times as long to get there):

In [52]:
%time show((4, 4, 4, 4), ops=OPS + "⌊⌈")

Can make 0 to 1644 with expressions((4, 4, 4, 4), "+-*/^_√!.$⌊⌈"). (1,147,507 table entries)

   0 = 44-44
   1 = 44/44
   2 = √⌊4.444⌋
   3 = 4-⌈.444⌉
   4 = ⌊4.444⌋
   5 = ⌈4.444⌉
   6 = √(44-(4+4))
   7 = (44/4)-4
   8 = 4+⌊4.44⌋
   9 = 4+⌈4.44⌉
  10 = 44/4.4
  11 = 44/⌊4.4⌋
  12 = (4+44)/4
  13 = √4+(44/4)
  14 = 4+(4+(4+√4))
  15 = 4+(44/4)
  16 = .4*(44-4)
  17 = ⌊(4*4.44)⌋
  18 = ⌈(4*4.44)⌉
  19 = ⌊(.44*44)⌋
  20 = 4*⌈4.44⌉
  21 = (4+4.4)/.4
  22 = √4*(44/4)
  23 = 4!-⌈.444⌉
  24 = 4+(4+(4*4))
  25 = ⌊(.444^-4)⌋
  26 = 4+(44/√4)
  27 = ⌈(4*√44.4)⌉
  28 = 44-(4*4)
  29 = 4!+⌈4.44⌉
  30 = (4+(4+4))/.4
  31 = 4+⌈(4*√44)⌉
  32 = (4*4)+(4*4)
  33 = ⌊(4*(4+4.4))⌋
  34 = 44-(4/.4)
  35 = 4!+(44/4)
  36 = 44-(4+4)
  37 = 44-⌈√44⌉
  38 = 44-(4+√4)
  39 = 44-⌈4.4⌉
  40 = 44-⌊4.4⌋
  41 = ⌈44.4⌉-4
  42 = √4+(44-4)
  43 = 44-(4/4)
  44 = ⌊44.44⌋
  45 = ⌈44.44⌉
  46 = 4+(44-√4)
  47 = √4+⌈44.4⌉
  48 = 4*(4+(4+4))
  49 = 4+⌈44.4⌉
  50 = 4+(√4+44)
  51 = 44+⌈√44⌉
  52 = 4+(4+44)
  53 = ⌊((4+4)*

# Even More Fives: Five 5s

In the [xkcd forum](http://forums.xkcd.com/viewtopic.php?f=14&t=116813&start=280) they took up the problem of **five 5s** and got all the integers up to 298, using the "double factorial" and π functions. We can get up to 171, using just the default operators, but with five digits it does take about seven minutes, whereas all the other puzzles with four or three digits (and without floor and ceiling) took less than a minute. I suspect you could go much further using floor and ceiling, but that computation would take even longer, so for now let's stick with our default set of operations:

In [53]:
%time show((5, 5, 5, 5, 5))

Can make 0 to 171 with expressions((5, 5, 5, 5, 5), "+-*/^_√!.$"). (7,624,387 table entries)

   0 = 5*(55-55)
   1 = 5^(55-55)
   2 = 55/(.5*55)
   3 = .5*((55/5)-5)
   4 = 5-(55/55)
   5 = 5+(55-55)
   6 = 5+(55/55)
   7 = ((5+55)/5)-5
   8 = .5*(5+(55/5))
   9 = 5!-(555/5)
  10 = 5!-(55+55)
  11 = 5*(55/(5*5))
  12 = (5/5)+(55/5)
  13 = (5+(5+55))/5
  14 = (5*5)-(55/5)
  15 = 5+(55/5.5)
  16 = (55+(5*5))/5
  17 = 5+((5+55)/5)
  18 = 5+((5!-55)/5)
  19 = (5*5)-(5+(5/5))
  20 = 55/(5*.55)
  21 = 5+(5+(55/5))
  22 = (55+55)/5
  23 = (5+(55/.5))/5
  24 = (5-(55/55))!
  25 = 55-(5+(5*5))
  26 = 5*(5+(5/(5*5)))
  27 = 5+(55/(.5*5))
  28 = .5*(55+(5/5))
  29 = 5+((5*5)-(5/5))
  30 = 5*((55/5)-5)
  31 = 5+((5/5)+(5*5))
  32 = (5+(55/5))/.5
  33 = .55*(5+55)
  34 = (5!+(55-5))/5
  35 = 5+(55-(5*5))
  36 = (5*5)+(55/5)
  37 = 5+((5.5-5)^-5)
  38 = .5+(5*(5+(.5*5)))
  39 = ((5*5)-5.5)/.5
  40 = 55-(5+(5+5))
  41 = (5!*.55)-(5*5)
  42 = (5+5.5)/(.5*.5)
  43 = 55-(5!/(5+5))
  44 = 55-(55/5)
  45

# What's Next?

One exercise would be adding even more operators, such as:

- **Nth root**: `3√8` = 2
- **Percent**: `5%` = 5/100
- **Repeating decimal**: `.4...` = .44444444... = 4/9
- **Double factorial**: `9!!` = 9 × 7 × 5 × 3 × 1 = 945; not the same as `(9!)!`
- **Gamma function**: `Γ(n)` = (n − 1)! and works for non-integers
- **Prime counting function**: `π(n)` = number of primes ≲ n; e.g. `π(5)` = 3
- **Transcendental functions**: `log`, `sin` `cos`, `tan`, `arcsin`, ... maybe degree symbol: 90°
- **Matrix notation**: with determinant symbol to get a number.
- **Combinations and Permutations**: `n P k` and `n C k`

Another approach would be to look around for related puzzles and solve them. What do you want to do?